# Class definition

In [1]:
class LinearRegression:
    """
    OLS Linear Regression.

    LinearRegression fits a linear model with k regressors, with fitted coefficients 
    b = (b1, ..., bk) to minimize the residual sum of squares between the observed 
    target variable in the dataset, and the target predicted by the linear approximation.
    
    Needed packages:
    import numpy as np
    from scipy import stats

    Parameters
    ----------
    fit_intercept : bool, default=True
        Whether to calculate the intercept for this model. If set to False, 
        no intercept will be used in calculations (e.g. use if data is centered).
        
    Attributes
    ----------
    coefficients : array of shape k+1: #features/regressors + 1
        Estimated coefficients for the linear regression problem.
        This includes the intercept, as first value in the array.
        
    intercept : array of shape 1
    Independent term/constant in the linear model. Set to 0.0 if fit_intercept = False.
    
    residuals : array of shape N
        Estimated residuals, defined as the difference between the predicted,
        and the true y-value
        
    se_coefficients : array of shape k+1
        Estimated standard errors of estimated regression coefficients.
    
    t_values : array of shape k+1
        Estimated t-values of estimated regression coefficients, for H0: b=0.
    
    p_values : array of shape k+1
        Estimated p-values of estimated regression coefficients, for H0: b=0.

    n_features : int
        Number of features seen during method `fit`, excluding intercept.
        

        
    Methods
    ----------
    fit(X, y): 
        Fit linear model, and create attributes
    
    predict(X): 
        Predict target variable, using the fitted parameters of this estimator.
    
    R_squared(X,y): 
        Return the coefficient of determination of the prediction.
    
    adjusted_R_squared(X,y): 
        Return the adjusted coefficient of determination of the prediction.
    
    summary(decimals): 
        Print summary of regression output after fit.
    

    Examples
    --------
    >>> import numpy as np
    >>> X = np.array([[1, 1], [1, 2], [2, 2], [2, 3]])
    >>> y = np.array([[0], [4], [0], [6]])
    >>> regfit = LinearRegression(fit_intercept=True).fit(X, y)
    >>> regfit.summary()
    Regression output:
               Coefficient  S.E.       t-value    p-value   
    Intercept  -1.5         1.658      -0.905     0.532     
    X1         -4.0         1.414      -2.828     0.216     
    X2         5.0          1.0        5.0        0.126     

    Residuals:
    [-0.5  0.5  0.5 -0.5]

    R-squared: 0.963               	 Adjusted R-squared: 0.889
    
    """
    def __init__(
        self, 
        fit_intercept=True,
    ):
        self.fit_intercept = fit_intercept
        

    def fit(self, X, y):
        """
        Fit linear regression model, simple OLS.

        Parameters
        ----------
        X : array of shape Nxk (Training data)
        y : array of shape N (Target values)

        Returns
        -------
        self : object
            Fitted Estimator.
        """
        # Define n_features and n_observations
        self.n_observations = len(y)
        self.n_features = len(X[0])
        
        # Compute mean and sd of dependent variable
        self.ymean = y.mean()
        self.ystd = y.std()
        
        # Compute coefficients
        if self.fit_intercept == True:
            X = np.append(np.ones((len(X),1)), X, axis=1)
            self.coefficients = (np.linalg.inv(X.T@X)@np.transpose(X)@y).T
            self.intercept = self.coefficients[0]
        else:
            self.coefficients = (np.linalg.inv(X.T@X)@np.transpose(X)@y).T
            self.intercept = 0.0
            
        # Compute residuals
        self.y_predicted = self.coefficients @ X.T
        self.residuals = y - self.y_predicted.T
        
        # Compute standard error of coefficients and of regression
        var_hat = (self.residuals.T @ self.residuals) / (len(y) - self.n_features - 1)
        self.se_coefficients = np.sqrt(np.diag(var_hat * np.linalg.inv(X.T@X)))
        self.se_regression = np.sqrt(var_hat)
        
        # Compute t- and p-values for Null hypothesis H0: b=0
        self.t_values = self.coefficients / self.se_coefficients
        if self.fit_intercept == True:
            df = len(X) - len(X[0])
        else:
            df = len(X) - len(X[0]) - 1
        self.p_values = 2*(stats.t.sf(abs(self.t_values), df))
        
        # Compute 95% CI
        self.lowerCI = self.coefficients - stats.t.isf(0.025, self.n_observations-self.n_features-1) * self.se_coefficients
        self.upperCI = self.coefficients + stats.t.isf(0.025, self.n_observations-self.n_features-1) * self.se_coefficients
        
        # Compute SSR, SST, SSE
        self.SST = np.sum((y-y.mean())**2)
        self.SSR = np.sum(self.residuals**2)
        self.SSE = self.SST - self.SSR
        
        # Compute llf
        self.llf = -np.log(self.SSR)*(self.n_observations/2) - (1+np.log(np.pi/(self.n_observations/2)))*(self.n_observations / 2)
        
        # Compute selection criteria RMSE, MAE, AIC, BIC, HQIC
        self.RMSE = np.sqrt(np.mean((y - self.y_predicted)**2))
        self.MAE = np.mean(abs(y - self.y_predicted))
        self.AIC = -2 * self.llf + 2 * (self.n_features + 1)
        self.BIC = -2 * self.llf + np.log(self.n_observations) * (self.n_features + 1)
        self.HQIC = -2 * self.llf + 2 * (self.n_features + 1) * np.log(np.log(self.n_observations))
        
        # Compute F-statistic
        self.F_statistic = (self.SSE/(self.n_features))/ (self.SSR/(self.n_observations - self.n_features - 1))
        self.F_statistic_p_value = stats.f.sf(self.F_statistic, self.n_features, self.n_observations-self.n_features-1)
        
        # Compute residual skewness, kurtosis
        self.res_skewness = np.mean(self.residuals**3) / (np.mean(self.residuals**2)**1.5)
        self.res_kurtosis = np.mean(self.residuals**4) / (np.mean(self.residuals**2)**2)
        
        # Compute Durbin-Watson test statistic
        self.diff_residuals = np.diff(self.residuals, axis=0)
        self.DW_statistic = np.sum(self.diff_residuals**2) / np.sum(self.residuals**2)
        
        # Compute JB test statistic and p-value
        self.JB_statistic = len(self.residuals) / 6 * (self.res_skewness**2 + (self.res_kurtosis - 3)**2 / 4)
        self.JB_statistic_p_value = stats.chi2.sf(self.JB_statistic, 2)
        
        return self
    
    def predict(self, X):
        """
        Predict using the fitted parameters of this Linear Regression estimator.

        Parameters
        ----------
        X : array of shape M, k-1
            M values for k-1 regressors test data. Don't add 1 for the intercept.

        Returns
        -------
        C : array of shape M
            Returns predicted value.
        """
        if hasattr(self, 'coefficients') == False:
                raise ValueError(
                     " This LinearRegression instance is not fitted yet." \
                     " Call 'fit' method with appropriate X and y before using this predict function."
                 )
                
        if self.fit_intercept == True:
            X = np.vstack([np.ones(len(X)), X.T]).T
        
        y_predicted = self.coefficients @ X.T

        return y_predicted.T
    
    def R_squared(self, X, y):
        """Return the coefficient of determination of the prediction.

        The coefficient of determination R^2 is defined as the residual
        sum of squares divided by the total sum of squares. The best possible R^2
        score is 1.0, explaining all variance in the data. The R^2 score can be negative
        when no constant is included, or the model is made arbitrarily bad. A model 
        always predicting the mean of y, the expected value of y, disregarding the input 
        features, gets a R^2 score of 0.0.

        Parameters
        ----------
        X : array of shape Nxk
        y : array of shape N


        Returns
        -------
        R_squared : float
            R^2 score of predicted X wrt. true y
        """
        
        if hasattr(self, 'coefficients') == False:
                raise ValueError(
                     " This LinearRegression instance is not fitted yet." \
                     " Call 'fit' method with appropriate X and y before using this score function."
                 )
        
        SSR = sum((y - self.predict(X))**2)
        SST = sum((y - y.mean())**2)
        R_squared = (1 - SSR/SST)
        
        return R_squared
    
    def adjusted_R_squared(self, X, y):
        """Return the adjusted coefficient of determination of the prediction.

        The adjusted coefficient of determination R^2 is defined as the residual
        sum of squares divided by N-k, divided by the total sum of squares, divided by N-1. 
        This allows penalty for #regressors.

        Parameters
        ----------
        X : array of shape Nxk
        y : array of shape N


        Returns
        -------
        adjusted_R_squared : float
            Adjusted R^2 score of predicted X wrt. true y
        """
        
        if hasattr(self, 'coefficients') == False:
                raise ValueError(
                     " This LinearRegression instance is not fitted yet." \
                     " Call 'fit' method with appropriate X and y before using this score function."
                 )
        
        N = len(X)
        k = len(X[0])
        
        R2 = self.R_squared(X,y)
        adjusted_R_squared = 1 - ((N-1)/(N-k-1))* (1 - R2)
        
        return adjusted_R_squared
    
    
    def summary(self, decimals=3):
        """Returns the table of the regression output.

        Parameters
        ----------
        decimals : int, default=3
            Number of decimals to round to in the table.


        Returns
        -------
        regression_table : table
            Table of the regression output.
        """
        
        if hasattr(self, 'coefficients') == False:
                raise ValueError(
                     " This LinearRegression instance is not fitted yet." \
                     " Call 'fit' method with appropriate X and y before using this predict function."
                 )

        print('Regression output:')
        table = np.vstack([self.coefficients, 
                        self.se_coefficients,
                        self.t_values, 
                        self.p_values]).T

        regression_dict = dict()
        for num in range(len(self.coefficients)):
            if num == 0:
                if self.fit_intercept == True:
                    regression_dict['Intercept'] = table[num]
                else:
                    regression_dict['X1'] = table[num]
            else:
                if self.fit_intercept == True:
                    string = f"X{num}"
                    regression_dict[string] = table[num]
                else:
                    string = f"X{num+1}"
                    regression_dict[string] = table[num]
        
        if decimals < 4:
            L = 10
        else:
            L = decimals + 7
            
                
        print("{:<{L}} {:<{L2}} {:<{L}} {:<{L}} {:<{L}}".format('', 'Coefficient', 'S.E.', 't-value', 'p-value', 
                                                                L=L, L2=L+2))
        
        for k, v in regression_dict.items():
            estim, se, tval, pval = v
            print("{:<{L}} {:<{L2}} {:<{L}} {:<{L}} {:<{L}}".format(k, 
                                                              np.round(estim, decimals), 
                                                              np.round(se, decimals), 
                                                              np.round(tval, decimals),
                                                              np.round(pval,decimals), L=L, L2=L+2))

        print('\n(First 5) Residuals:')
        print(self.residuals[:5])

        print(f'\nR-squared: {np.round(self.R_squared(X,y), decimals)} \
              \t Adjusted R-squared: {np.round(self.adjusted_R_squared(X,y), decimals)}')
        
        
    def html_summary(self):
        """Returns HTML table of the regression output.

        Returns
        -------
        regression_table : table
            Table of the regression output.
        """
        
        dt = datetime.now()
        
        variable_names = []
        for num in range(len(self.coefficients)):
            if num == 0:
                if self.fit_intercept == True:
                    variable_names.append('Intercept')
                else:
                    variable_names.append('X1')
            else:
                if self.fit_intercept == True:
                    string = f"X{num}"
                else:
                    string = f"X{num+1}"
                variable_names.append(string)
                    
        table = np.vstack([["Variable", "Coefficient", "Std. error", "t-stat", "p-value"], 
                           np.vstack([variable_names,
                                      self.coefficients, 
                                      self.se_coefficients,
                                      self.t_values, 
                                      self.p_values]).T])
        
        regression_table = tabulate(table, headers="firstrow", tablefmt='html')
        
        text = HTML(f"""
        <h2>Linear Regression Results</h2>

        <p> </p>
        <pre> Dependent variable: y </pre>
        <pre> Method: Least squares </pre>
        <pre> Date: {dt.month}/{dt.day}/{dt.year}, Time: {dt.hour}:{dt.minute}</pre>
        <pre> Observations: {self.n_observations} </pre>
        <pre> Variables (excluding intercept): {self.n_features} </pre>
        <pre> D.o.f. Residuals: {self.n_observations - self.n_features - 1}</pre>
        <pre> D.o.f. Model: {self.n_features} </pre>

        {regression_table}

        <table>
            <thead>
                <tr><th>Criterions and statistics   </th></tr>
            </thead>
            <tr>
                <td>R-squared</td>
                <td>{self.R_squared(X,y):6.6f}</td>
                <td></td>        
                <td></td>
                <td>Mean dependent var.</td>
                <td>{self.ymean:6.6f}</td>
                <td></td>        
                <td></td>
                <td>Durbin-Watson statistic</td>
                <td>{self.DW_statistic:6.6f}</td>
            </tr>
            <tr>
                <td>Adjusted R-squared</td>
                <td>{self.adjusted_R_squared(X,y):6.6f}</td>
                <td></td>        
                <td></td>
                <td>SD dependent var.</td>
                <td>{self.ystd:6.6f}</td>
                <td></td>        
                <td></td>
                <td>Jarque-Bera statistic</td>
                <td>{self.JB_statistic:6.6f}</td>
            </tr>
            <tr>
                <td>S.E. of regression</td>
                <td>{self.se_regression:6.6f}</td>
                <td></td>        
                <td></td>
                <td>Akaike information crit.</td>
                <td>{self.AIC:6.6f}</td>
                <td></td>        
                <td></td>
                <td>Prob(Jarque-Bera)</td>
                <td>{self.JB_statistic_p_value:6.6f}</td>
            </tr>
            <tr>
                <td>Sum squared residuals</td>
                <td>{self.SSR:6.6f}</td>
                <td></td>        
                <td></td>
                <td>Bayesian information crit.</td>
                <td>{self.BIC:6.6f}</td>
                <td></td>        
                <td></td>
                <td>Skewness residuals</td>
                <td>{self.res_skewness:6.6f}</td>
            </tr>
            <tr>
                <td>Log Likelihood</td>
                <td>{self.llf:6.6f}</td>
                <td></td>        
                <td></td>
                <td>Hanann-Quin information crit.</td>
                <td>{self.HQIC:6.6f}</td>
                <td></td>        
                <td></td>
                <td>Kurtosis residuals</td>
                <td>{self.res_kurtosis:6.6}</td>
            </tr>
            <tr>
                <td>F-statistic</td>
                <td>{self.F_statistic:6.6}</td>
                <td></td>        
                <td></td>
            </tr>
            <tr>
                <td>Prob(F-statistic)</td>
                <td>{self.F_statistic_p_value:6.6}</td>
                <td></td>        
                <td></td>
            </tr>
        </table>
        """
        )

        display(text)



## Examples

In [2]:
import numpy as np
from scipy import stats
from IPython.display import display, HTML
from datetime import datetime
from tabulate import tabulate
import sklearn.datasets
 
data, target = sklearn.datasets.load_iris(return_X_y=True, as_frame=True)
X = data.values
y = target.values

regfit = LinearRegression(fit_intercept=False).fit(X, y)

In [3]:
regfit.summary(3)

Regression output:
           Coefficient  S.E.       t-value    p-value   
X1         -0.084       0.049      -1.714     0.089     
X2         -0.024       0.057      -0.413     0.68      
X3         0.225        0.057      3.955      0.0       
X4         0.6          0.094      6.37       0.0       

(First 5) Residuals:
[0.07861541 0.04993583 0.06023686 0.00445714 0.07252236]

R-squared: 0.93               	 Adjusted R-squared: 0.928


In [4]:
regfit.html_summary()